In [15]:
import numpy as np
import scipy.stats as stats
from math import log, sqrt, exp
from scipy.stats import norm
import sys
sys.path.append('/Users/ansel_li/Fintech545/public/')
from risk_mgmt import black_s, calc_return, VaR
from scipy.optimize import minimize
import pandas as pd
from datetime import datetime
from scipy.stats import t, norm, spearmanr, multivariate_normal
from datetime import datetime, timedelta


In [5]:
# Input parameters
S = 151.03  # Current stock price
K = 165  # Strike price
t = (np.datetime64('2022-03-13') - np.datetime64('2022-03-13')).astype(int) / 365.0
T = (np.datetime64('2022-04-15') - np.datetime64('2022-03-13')).astype(int) / 365.0
r = 0.0425  # Risk-free interest rate
q = 0.0053  # Continuously compounding dividend yield
sigma = 0.20  # Implied volatility


In [6]:
def gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call'):
    tau = T - t
    d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
    d2 = d1 - sigma * sqrt(tau)

    N_d1 = stats.norm.cdf(d1)
    N_d2 = stats.norm.cdf(d2)

    if option_type == 'call':
        delta = exp(-q * tau) * N_d1
        gamma = exp(-q * tau) * N_d1 / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * N_d1 * sigma / (2 * np.sqrt(tau)) - r * K * np.exp(-r * tau) * -N_d2 + q * S * np.exp(-q * tau) * -N_d1
        rho = K * tau * np.exp(-r * tau) * norm.cdf(d2)
    elif option_type == 'put':
        delta = -exp(-q * tau) * (1 - N_d1)
        gamma = exp(-q * tau) * N_d1 / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * N_d1 * sigma / (2 * np.sqrt(tau)) + r * K * np.exp(-r * tau) * -N_d2 - q * S * np.exp(-q * tau) * -N_d1
        rho = -K * tau * np.exp(-r * tau) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return delta, gamma, vega, theta, rho

In [7]:
call_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call')
put_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks)
print("Put Greeks: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks)

Call Greeks: Delta, Gamma, Vega, Theta, Rho
(0.08297130333914773, 0.009135328237370505, 6.938710929513443, -3.7196562346632, 1.1025939156368187)
Put Greeks: Delta, Gamma, Vega, Theta, Rho
(-0.9165496333661425, 0.009135328237370505, 6.938710929513443, -4.623431322046896, -13.758003122735788)


In [8]:
def finite_difference(S, K, t, T, r, q, sigma, option_type='call', delta_shift=0.01):
    S_up = S + S * delta_shift
    S_down = S - S * delta_shift
    sigma_up = sigma + delta_shift
    sigma_down = sigma - delta_shift
    r_up = r + delta_shift
    r_down = r - delta_shift

    def option_price(S, K, t, T, r, q, sigma, option_type):
        tau = T - t
        d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
        d2 = d1 - sigma * sqrt(tau)

        if option_type == 'call':
            return S * exp(-q * tau) * stats.norm.cdf(d1) - K * exp(-r * tau) * stats.norm.cdf(d2)
        elif option_type == 'put':
            return K * exp(-r * tau) * stats.norm.cdf(-d2) - S * exp(-q * tau) * stats.norm.cdf(-d1)

    # Finite difference calculations
    delta_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - option_price(S_down, K, t, T, r, q, sigma, option_type)) / (2 * S * delta_shift)
    gamma_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - 2 * option_price(S, K, t, T, r, q, sigma, option_type) + option_price(S_down, K, t, T, r, q, sigma, option_type)) / (S * delta_shift)**2
    vega_fd = (option_price(S, K, t, T, r, q, sigma_up, option_type) - option_price(S, K, t, T, r, q, sigma_down, option_type)) / (2 * 0.01)
    theta_fd = - (option_price(S, K, t + 1/365, T, r, q, sigma, option_type) - option_price(S, K, t, T, r, q, sigma, option_type)) / (-1/365)
    rho_fd = (option_price(S, K, t, T, r_up, q, sigma, option_type) - option_price(S, K, t, T, r_down, q, sigma, option_type)) / (2 * 0.01)

    return delta_fd, gamma_fd, vega_fd, theta_fd, rho_fd

In [9]:
call_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='call')
put_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks_fd)
print("Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks_fd)

import pandas as pd

# Combine the results in a dictionary
greek_data = {
    "Call Closed-form": call_greeks,
    "Call Finite Diff.": call_greeks_fd,
    "Put Closed-form": put_greeks,
    "Put Finite Diff.": put_greeks_fd
}

# Create a DataFrame
greeks_comparison = pd.DataFrame(greek_data, index=["Delta", "Gamma", "Vega", "Theta", "Rho"])

# Display the DataFrame
print(greeks_comparison)

Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(0.08390256513620564, 0.016848978828828364, 6.932942300084921, -8.048047189743537, 1.1026982009081365)
Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(-0.9156183715690898, 0.016848978828841604, 6.9329423000851875, -1.8621154051211875, -13.757900862010786)
       Call Closed-form  Call Finite Diff.  Put Closed-form  Put Finite Diff.
Delta          0.082971           0.083903        -0.916550         -0.915618
Gamma          0.009135           0.016849         0.009135          0.016849
Vega           6.938711           6.932942         6.938711          6.932942
Theta         -3.719656          -8.048047        -4.623431         -1.862115
Rho            1.102594           1.102698       -13.758003        -13.757901


In [10]:
def binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r) * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Create the stock price tree
    stock = np.zeros((n + 1, n + 1))
    stock[0, 0] = S

    for i in range(1, n + 1):
        stock[i, 0] = stock[i - 1, 0] * u
        for j in range(1, i + 1):
            stock[i, j] = stock[i - 1, j - 1] * d

    # Adjust stock prices for dividends
    if div_date is not None and exercise_type == 'american':
        div_steps = int((div_date - np.datetime64('2022-03-13')).astype(int) / 365.0 * n)
        stock[div_steps:] -= div_amt

    # Create the option price tree
    option = np.zeros_like(stock)

    # Calculate the option's value at maturity
    if option_type == 'call':
        option[:, -1] = np.maximum(stock[:, -1] - K, 0)
    elif option_type == 'put':
        option[:, -1] = np.maximum(K - stock[:, -1], 0)

    # Step back through the tree to calculate the option's value at each node
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option[i, j] = discount * (p * option[i + 1, j] + (1 - p) * option[i + 1, j + 1])

            # Calculate the option's value if exercised early
            if exercise_type == 'american':
                if option_type == 'call':
                    early_exercise = np.maximum(stock[i, j] - K, 0)
                elif option_type == 'put':
                    early_exercise = np.maximum(K - stock[i, j], 0)

                option[i, j] = np.maximum(option[i, j], early_exercise)

    return option




def binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0, delta_shift=0.01, rate_shift=0.0001, vol_shift=0.01):
    option_tree = binomial_tree(S, K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_up = binomial_tree(S * (1 + delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_down = binomial_tree(S * (1 - delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_up = binomial_tree(S, K, T, r, sigma + vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_down = binomial_tree(S, K, T, r, sigma - vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_up = binomial_tree(S, K, T, r + rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_down = binomial_tree(S, K, T, r - rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)

    # Calculate Greeks
    delta = (option_tree_S_up[0, 0] - option_tree_S_down[0, 0]) / (2 * S * delta_shift)
    gamma = (option_tree_S_up[0, 0] - 2 * option_tree[0, 0] + option_tree_S_down[0, 0])/ (S * delta_shift) ** 2
    theta = -(option_tree[0, 0] - binomial_tree(S, K, T - 1/n, r, sigma, n, option_type, exercise_type, div_date, div_amt)[0, 0]) / (1/n)

    vega = (option_tree_vol_up[0, 0] - option_tree_vol_down[0, 0]) / (2 * vol_shift)
    rho = (option_tree_rate_up[0, 0] - option_tree_rate_down[0, 0]) / (2 * rate_shift)

    return delta, gamma, theta, vega, rho



In [11]:
n=100
div_date = np.datetime64('2022-04-11')
div_amt = 0.88  # Dividend amount

call_without_div = binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american')[0, 0]
put_without_div = binomial_tree(S, K, T, r, sigma, n, option_type='put', exercise_type='american')[0, 0]
call_with_div = binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=div_amt)[0, 0]
put_with_div = binomial_tree(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=div_amt)[0, 0]

print("American Call Option without Dividends:", call_without_div)
print("American Put Option without Dividends:", put_without_div)
print("American Call Option with Dividends:", call_with_div)
print("American Put Option with Dividends:", put_with_div)

American Call Option without Dividends: 0.3317910184978169
American Put Option without Dividends: 14.016650966102587
American Call Option with Dividends: 0.27365120446834945
American Put Option with Dividends: 14.86187574483816


In [12]:
# calculate greeks
call_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american')
put_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american')
call_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=div_amt)
put_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=div_amt)

# create dataframe for greeks
greek_labels = ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']
call_greeks = pd.DataFrame({'Call without div': call_greeks_without_div, 'Call with div': call_greeks_with_div}, index=greek_labels)
put_greeks = pd.DataFrame({'Put without div': put_greeks_without_div, 'Put with div': put_greeks_with_div}, index=greek_labels)

# display greeks dataframes
print('Call Option Greeks:')
print(call_greeks)

print('\nPut Option Greeks:')
print(put_greeks)

Call Option Greeks:
       Call without div  Call with div
Delta          0.082067       0.072438
Gamma          0.016219       0.014888
Theta         -7.663421      -6.868668
Vega           7.031265       5.894682
Rho            1.080959       0.917994

Put Option Greeks:
       Put without div  Put with div
Delta        -0.957268     -0.964653
Gamma         0.020478      0.014834
Theta        -2.258358     -1.720999
Vega          3.751046      2.939957
Rho          -2.847535     -2.823484


In [13]:
# Calculate the sensitivity of the call and put to a change in dividend amount
Delta_call_dividend = (call_with_div - call_without_div) / div_amt
Delta_put_dividend = (put_with_div - put_without_div) / div_amt

print("Delta Call with Dividend: ", Delta_call_dividend)
print("Delta Put with Dividend: ", Delta_put_dividend)

Delta Call with Dividend:  -0.06606797048803119
Delta Put with Dividend:  0.9604827031086066


In [14]:
def max_sharpe_ratio_weights(cov_m, exp_returns, rf):
    num_stocks = len(exp_returns)
    
    # Define the Sharpe Ratio objective function to be minimized
    def neg_sharpe_ratio(weights):
        port_return = np.dot(weights, exp_returns)
        port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_m, weights)))
        sharpe_ratio = (port_return - rf) / port_volatility
        return -sharpe_ratio
    
    # Define the constraints
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # The sum of the weights must be 1
    bounds = tuple([(0, 1) for i in range(num_stocks)]) # The weights must be between 0 and 1
    
    # Find the portfolio weights that maximize the Sharpe Ratio
    initial_weights = np.ones(num_stocks) / num_stocks # Start with equal weights
    opt_results = minimize(neg_sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_results.x.round(4), opt_results.fun 


In [15]:
# Read all data
ff3 = pd.read_csv("F-F_Research_Data_Factors_daily.CSV")
mom = pd.read_csv("F-F_Momentum_Factor_daily.CSV")
# prices = pd.read_csv("DailyPrices.csv")
# returns = calc_return.return_calculate(prices, date_column="Date")
rf = 0.0025
returns = pd.read_csv('DailyReturn.csv')
# Join the FF3 data with the Momentum Data
ffData = pd.merge(ff3, mom, on="Date")
ffData.rename(columns={ffData.columns[-1]: "Mom"}, inplace=True)
ffData.iloc[:, 1:] = ffData.iloc[:, 1:].div(100)
ffData['Date'] = pd.to_datetime(ffData['Date'].astype(str), format="%Y%m%d")

returns['Date'] = pd.to_datetime(returns['Date'], format="%m/%d/%Y")
# Our 20 stocks
# stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']
stocks = ['AAPL', 'MSFT', 'BRK-B', 'CSCO', 'JNJ']
# Data set of all stock returns and FF3+1 returns
to_reg = pd.merge(returns[['Date'] + stocks], ffData, on="Date")

xnames = ["Mkt-RF", "SMB", "HML", "Mom"]

# OLS Regression for all Stocks
X = np.column_stack((np.ones(len(to_reg)), to_reg[xnames].values))
Y = to_reg[stocks].values

Betas = np.linalg.inv(X.T @ X) @ X.T @ Y
Betas = Betas.T

# Calculate the means of the last 10 years of factor returns
# adding the 0.0 at the front to 0 out the fitted alpha in the next step
means = np.concatenate(([0.0], ffData.loc[ffData['Date'] >= datetime(2013, 2, 9), xnames].mean()))

# Discrete Returns, convert to Log Returns and scale to 1 year
stockMeans = np.log(1 + Betas @ means) * 255 + rf
covar = np.cov(np.log(1.0 + Y.T)) * 255


In [16]:
weights, sharpe = max_sharpe_ratio_weights(covar, stockMeans, rf)
print("Sharpe ratio is:", -sharpe)

Sharpe ratio is: 1.002075889964526


In [17]:
portfolio_weights = np.array(weights)
portfolio_weights

array([0.1013, 0.223 , 0.3998, 0.0789, 0.1969])

In [18]:
expected_return = portfolio_weights.dot(stockMeans)
print(expected_return)

0.11751080534238349


# Problem 1
part2

In [14]:
# Get Updated Prices
stocks = ['AAPL', 'MSFT', 'BRK-B', 'CSCO', 'JNJ']
portfolio_weights = np.array([0.1013, 0.223 , 0.3998, 0.0789, 0.1969])
updated = pd.read_csv("/Users/ansel_li/FinTech-545-Spring2023/Week08/updated_prices.csv")
updated["Date"] = pd.to_datetime(updated.Date, format="%m/%d/%Y")
upReturns = calc_return.return_calculate(updated, date_column="Date")

# Calculate portfolio return and updated weights for each day
n = upReturns.shape[0]
m = len(stocks)

pReturn = np.empty(n)
weights = np.empty((n, len(portfolio_weights)))
lastW = portfolio_weights.copy()
matReturns = upReturns[stocks].values
print(matReturns)
for i in range(n):
    # Save Current Weights in Matrix
    weights[i, :] = lastW

    # Update Weights by return
    lastW = lastW * (1.0 + matReturns[i, :])

    # Portfolio return is the sum of the updated weights
    pR = lastW.sum()

    # Normalize the weights back so sum = 1
    lastW = lastW / pR

    # Store the return
    pReturn[i] = pR - 1
# Set the portfolio return in the Update Return DataFrame
upReturns["Portfolio"] = pReturn

# Calculate the total return
totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1

# Calculate the Carino K
k = np.log(totalRet + 1) / totalRet

# Carino k_t is the ratio scaled by 1/K
carinoK = np.log(1.0 + pReturn) / pReturn / k

# Calculate the return attribution
attrib = pd.DataFrame(matReturns * weights * carinoK[:, np.newaxis], columns=stocks)

# Set up a DataFrame for output
Attribution = pd.DataFrame({"Value": ["TotalReturn", "Return Attribution"]})

# Loop over the stocks
for s in stocks + ["Portfolio"]:
    # Total Stock return over the period
    tr = np.exp(np.sum(np.log(upReturns[s] + 1))) - 1

    # Attribution Return (total portfolio return if we are updating the portfolio column)
    atr = tr if s == "Portfolio" else attrib[s].sum()

    # Set the values
    Attribution[s] = [tr, atr]

# Realized Volatility Attribution

# Y is our stock returns scaled by their weight at each time
Y = matReturns * weights

# Set up X with the Portfolio Return
X = np.column_stack((np.ones(n), pReturn))

# Calculate the Beta and discard the intercept
B = np.linalg.inv(X.T @ X) @ X.T @ Y
B = B[1, :]

# Component SD is Beta times the standard Deviation of the portfolio
cSD = B * np.std(pReturn)

# Add the Vol attribution to the output
vol_attrib = pd.DataFrame({"Value": ["Vol Attribution"], **{stocks[i]: [cSD[i]] for i in range(len(stocks))}, "Portfolio": [np.std(pReturn)]})

Attribution = pd.concat([Attribution, vol_attrib], ignore_index=True)

print(Attribution)


[[-1.88941679e-02 -2.43392445e-02 -1.18470859e-02 -2.65645530e-02
  -4.40884054e-03]
 [-2.10247564e-02  2.24679386e-03 -1.72968524e-02 -1.38958312e-02
  -3.11196711e-03]
 [-1.03471555e-02 -5.70329297e-03 -1.18824146e-02 -1.39219011e-02
  -7.98415795e-03]
 [-1.27650431e-02 -1.84681547e-02 -1.86167931e-02 -2.41047168e-02
  -2.29956273e-03]
 [-4.86436920e-03  1.14851725e-03 -4.88169188e-03  3.88147495e-03
  -1.15242108e-02]
 [-1.13846517e-02 -2.65883829e-02  1.13916665e-02 -1.38840241e-02
   2.85942352e-02]
 [-5.63302820e-04  2.84931734e-02  6.73852660e-03 -1.39012473e-02
   4.47418541e-03]
 [-2.94313383e-03  1.05489995e-02 -5.14129518e-03 -1.28321521e-02
   1.31250009e-02]
 [ 6.97776174e-02  2.80817224e-02  1.69986699e-02  1.81252658e-02
   7.03435615e-03]
 [ 2.61257461e-02  8.82371531e-03  3.83493136e-04  1.07888867e-03
   2.91053112e-03]
 [-9.72658871e-04 -7.13873818e-03  3.00300950e-03 -5.38888110e-03
  -8.18384317e-03]
 [ 7.04429943e-03  1.52220984e-02  1.79641201e-02  1.44482394e-02

# Problem 2

In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Read CSV files
ff3 = pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
mom = pd.read_csv('F-F_Momentum_Factor_daily.CSV')
returns = pd.read_csv('DailyReturn.csv')

# Join the FF3 data with the Momentum Data
ffData = ff3.merge(mom, on='Date')
ffData.rename(columns={ffData.columns[-1]: 'Mom', 'Mkt-RF': 'Mkt_RF'}, inplace=True)
ffData.iloc[:, 1:] = ffData.iloc[:, 1:].values / 100
ffData['Date'] = pd.to_datetime(ffData['Date'], format='%Y%m%d')

returns['Date'] = pd.to_datetime(returns['Date'], format='%m/%d/%Y')

# Join the FF3+1 to Stock data - filter to stocks we want
stocks = ['AAPL', 'MSFT', 'BRK-B', 'CSCO', 'JNJ']
to_reg = returns[['Date', 'SPY'] + stocks].merge(ffData, on='Date')

xnames = ['Mkt_RF', 'SMB', 'HML', 'Mom']

# OLS Regression for all Stocks
X = np.hstack([np.ones((len(to_reg), 1)), to_reg[xnames].values])
Y = to_reg[stocks].values
Betas = (np.linalg.inv(X.T @ X) @ X.T @ Y).T[:, 1:]

max_dt = max(to_reg['Date'])
min_dt = max_dt - pd.DateOffset(years=10)
to_mean = ffData[(ffData['Date'] >= min_dt) & (ffData['Date'] <= max_dt)]

# Historic daily factor returns
exp_Factor_Return = to_mean[xnames].mean().values
expFactorReturns = pd.DataFrame({'Factor': xnames, 'Er': exp_Factor_Return})

# Scale returns and covariance to geometric yearly numbers
stockMeans = np.log(1 + Betas @ exp_Factor_Return) * 255
covar = np.cov(np.log(1 + Y), rowvar=False) * 255

def sr(w):
    _w = np.array(w)
    m = _w.T @ stockMeans - 0.0025
    s = np.sqrt(_w.T @ covar @ _w)
    return -(m / s)

n = len(stocks)
x0 = np.ones(n) / n
bounds = [(0, None) for _ in range(n)]
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

result = minimize(sr, x0, bounds=bounds, constraints=constraints)
w = result.x / np.sum(result.x)

OptWeights = pd.DataFrame({'Stock': stocks, 'Weight': w, 'cEr': stockMeans * w})
print(OptWeights)

# Get Updated Prices
updated = pd.read_csv("updated_prices.csv")
updated['Date'] = pd.to_datetime(updated['Date'], format='%m/%d/%Y')

# Define a function for calculating returns
def return_calculate(df, dateColumn="Date"):
    df = df.sort_values(by=dateColumn)
    returns = df.iloc[:, 1:].pct_change().dropna()
    returns[dateColumn] = df[dateColumn]
    return returns

upReturns = return_calculate(updated, dateColumn="Date")

upFf3 = pd.read_csv("updated_F-F_Research_Data_Factors_daily.CSV")
upMom = pd.read_csv("updated_F-F_Momentum_Factor_daily.CSV")
upFfData = upFf3.merge(upMom, on='Date')
upFfData.rename(columns={upFfData.columns[-1]: 'Mom', 'Mkt-RF': 'Mkt_RF'}, inplace=True)
upFfData[upFfData.columns[1:]] = upFfData[upFfData.columns[1:]] / 100
upFfData['Date'] = pd.to_datetime(upFfData['Date'], format='%Y%m%d')

upFfData = pd.concat([ffData, upFfData], ignore_index=True)
upFfData.drop(columns=['RF'], inplace=True)

# Filter the FF returns to just the Stock return data
upFfData = upReturns.merge(upFfData, on='Date')[['Date'] + xnames]

# Calculate portfolio return and updated weights for each day
n = len(upReturns)
m = len(stocks)

pReturn = np.zeros(n)
residReturn = np.zeros(n)
weights = np.zeros((n, len(w)))
factorWeights = np.zeros((n, len(xnames)))
lastW = w.copy()
matReturns = upReturns[stocks].values
ffReturns = upFfData[xnames].values

for i in range(n):
    # Save Current Weights in Matrix
    weights[i, :] = lastW

    # Factor Weight
    factorWeights[i, :] = np.sum(Betas * lastW[:, np.newaxis], axis=0)


    # Update Weights by return
    lastW = lastW * (1.0 + matReturns[i, :])

    # Portfolio return is the sum of the updated weights
    pR = np.sum(lastW)
    # Normalize the weights back so sum = 1
    lastW = lastW / pR
    # Store the return
    pReturn[i] = pR - 1

    # Residual
    residReturn[i] = (pR - 1) - factorWeights[i, :].T @ ffReturns[i, :]



# Set the portfolio return in the Update Return DataFrame
upFfData['Alpha'] = residReturn
upFfData['Portfolio'] = pReturn

# Calculate the total return
totalRet = np.exp(np.sum(np.log(pReturn + 1))) - 1
# Calculate the Carino K
k = np.log(totalRet + 1) / totalRet

# Carino k_t is the ratio scaled by 1/K
carinoK = np.log(1.0 + pReturn) / pReturn / k
# Calculate the return attribution
attrib = pd.DataFrame(ffReturns * factorWeights * carinoK[:, np.newaxis], columns=xnames)
attrib['Alpha'] = residReturn * carinoK

# Set up a DataFrame for output
Attribution = pd.DataFrame({'Value': ["TotalReturn", "Return Attribution"]})

newFactors = xnames + ['Alpha']
# Loop over the factors
for s in newFactors + ['Portfolio']:
    # Total Stock return over the period
    tr = np.exp(np.sum(np.log(upFfData[s] + 1))) - 1
    # Attribution Return (total portfolio return if we are updating the portfolio column)
    atr = tr if s == 'Portfolio' else attrib[s].sum()
    # Set the values
    Attribution[s] = [tr, atr]

# Check that the attribution sums back to the total Portfolio return
assert np.isclose(Attribution.loc[1, newFactors].sum(), totalRet)

# Realized Volatility Attribution

# Y is our stock returns scaled by their weight at each time
Y = np.hstack((ffReturns * factorWeights, residReturn[:, np.newaxis]))
# Set up X with the Portfolio Return
X = np.hstack((np.ones((len(pReturn), 1)), pReturn[:, np.newaxis]))
# Calculate the Beta and discard the intercept
B = (np.linalg.inv(X.T @ X) @ X.T @ Y)[1, :]
# Component SD is Beta times the standard deviation of the portfolio
cSD = B * np.std(pReturn)

# Check that the sum of component SD is equal to the portfolio SD
assert np.isclose(cSD.sum(), np.std(pReturn))

vol_attrib_df = pd.DataFrame(
    {"Value": "Vol Attribution", **{newFactors[i]: cSD[i] for i in range(len(newFactors))}, "Portfolio": np.std(pReturn)},
    index=[2]
)

Attribution = pd.concat([Attribution, vol_attrib_df], ignore_index=True)


print(Attribution)




   Stock    Weight       cEr
0   AAPL  0.100792  0.014285
1   MSFT  0.209487  0.035649
2  BRK-B  0.438283  0.048322
3   CSCO  0.081217  0.011085
4    JNJ  0.170222  0.012090
                Value    Mkt_RF       SMB       HML       Mom     Alpha  \
0         TotalReturn -0.058005 -0.001108  0.020274  0.008215  0.018044   
1  Return Attribution -0.044562  0.000103  0.002425 -0.000610  0.017580   
2     Vol Attribution  0.009775 -0.000109 -0.000715  0.000127  0.002640   

   Portfolio  
0  -0.025063  
1  -0.025063  
2   0.011718  


# P3

In [90]:
ff3 = pd.read_csv("F-F_Research_Data_Factors_daily.CSV")
mom = pd.read_csv("F-F_Momentum_Factor_daily.CSV")
returns = pd.read_csv("DailyReturn.csv")

ffData = pd.merge(ff3, mom, on="Date")
ffData.rename(columns={ffData.columns[-1]: "Mom", "Mkt-RF": "Mkt_RF"}, inplace=True)
ffData.iloc[:, 1:] = ffData.iloc[:, 1:] / 100
ffData["Date"] = pd.to_datetime(ffData["Date"], format="%Y%m%d")

returns["Date"] = pd.to_datetime(returns["Date"], format="%m/%d/%Y")

stocks = ["AAPL", "MSFT", "BRK-B", "CSCO", "JNJ"]
to_reg = pd.merge(returns[["Date", "SPY"] + stocks], ffData, on="Date")

xnames = ["Mkt_RF", "SMB", "HML", "Mom"]

X = np.column_stack((np.ones(len(to_reg)), to_reg[xnames].values))
Y = to_reg[stocks].values
Betas = np.linalg.inv(X.T @ X) @ X.T @ Y
resid = Y - X @ Betas.T
Betas = Betas[:, 1:]

max_dt = to_reg["Date"].max()
min_dt = max_dt - timedelta(days=365*10)
to_mean = ffData[(ffData["Date"] >= min_dt) & (ffData["Date"] <= max_dt)]

exp_Factor_Return = to_mean[xnames].mean().values
expFactorReturns = pd.DataFrame({"Factor": xnames, "Er": exp_Factor_Return})

stockMeans = np.log(1 + Betas @ exp_Factor_Return) * 255
covar = np.cov(np.log(1 + Y), rowvar=False) * 255

def pvol(w, covar):
    return np.sqrt(w.T @ covar @ w)

def pCSD(w, covar):
    pVol = pvol(w, covar)
    csd = w * (covar @ w) / pVol
    return csd

def sseCSD(w, covar):
    csd = pCSD(w, covar)
    mCSD = np.mean(csd)
    dCsd = csd - mCSD
    se = dCsd * dCsd
    return 1.0e5 * np.sum(se)

n = len(stocks)

cons = ({'type': 'eq', 'fun': lambda w: 1.0 - np.sum(w)})
bnds = [(0, None) for _ in range(n)]
initial_guess = np.array([1/n] * n)

res = minimize(sseCSD, initial_guess, args=(covar,), constraints=cons, bounds=bnds)
w = res.x

RPWeights = pd.DataFrame({"Stock": stocks, "Weight": w, "cEr": stockMeans * w, "CSD": pCSD(w, covar)})

m = Y.mean(axis=0)
Y = Y - m

models = [t.fit(Y[:, i]) for i in range(Y.shape[1])]
U = np.column_stack([(Y[:, i] - loc) / scale for i, (df, loc, scale) in enumerate(models)])


nSim = 5000

corsp = spearmanr(U).correlation
_simU = norm.cdf(multivariate_normal.rvs(mean=np.zeros(n), cov=corsp, size=nSim)).T
simReturn = np.empty_like(_simU)

for i in range(n):
    df, loc, scale = models[i]
    simReturn[:, i] = t.ppf(_simU[:, i], df=df, loc=loc, scale=scale)

def _ES(w):
    x = np.array(w)
    r = np.sum(simReturn * x[:, np.newaxis], axis=0)
    return VaR.ES(r)

def CES(w):
    x = np.array(w)
    n = len(x)
    ces = np.zeros(n)
    es = _ES(x)
    e = 1e-6
    for i in range(n):
        old = x[i]
        x[i] = x[i] + e
        ces[i] = old * (_ES(x) - es) / e
        x[i] = old
    return ces

def SSE_CES(w):
    ces = CES(w)
    ces = ces - np.mean(ces)
    return 1e3 * (ces.T @ ces)

res = minimize(SSE_CES, initial_guess, constraints=cons, bounds=bnds)
w = res.x

ES_RPWeights = pd.DataFrame({"Stock": stocks, "Weight": w, "cEr": stockMeans * w, "CES": CES(w)})
print(ES_RPWeights)
print(RPWeights)

for i, stock in enumerate(stocks):
    df, loc, scale = models[i]
    print(f"{stock} -- df: {df}, loc: {loc}, scale: {scale}")

# print(corsp)

   Stock    Weight       cEr       CES
0   AAPL  0.189243  0.000021  0.237646
1   MSFT  0.190263  0.027953  0.238429
2  BRK-B  0.208299 -0.021989  0.237438
3   CSCO  0.179592 -0.017450  0.237496
4    JNJ  0.232603  0.013840  0.238174
   Stock    Weight       cEr       CSD
0   AAPL  0.154203  0.000017  0.022688
1   MSFT  0.142566  0.020945  0.022688
2  BRK-B  0.265160 -0.027992  0.022688
3   CSCO  0.150786 -0.014651  0.022688
4    JNJ  0.287285  0.017094  0.022688
AAPL -- df: 1970111.246238185, loc: -4.696186663373325e-07, scale: 0.015911523598135014
MSFT -- df: 4.5964540434780785, loc: 0.00037631927018799157, scale: 0.01240097061462394
BRK-B -- df: 21.7484660281291, loc: -2.982870119172333e-05, scale: 0.008997359286131264
CSCO -- df: 4.542918915808315, loc: 0.0002855876803817062, scale: 0.010920213502832945
JNJ -- df: 9.428619560362845, loc: 9.002379518935252e-05, scale: 0.008225160760266727


In [91]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

ff3 = pd.read_csv("F-F_Research_Data_Factors_daily.CSV")
mom = pd.read_csv("F-F_Momentum_Factor_daily.CSV")
returns = pd.read_csv("DailyReturn.csv")

ffData = pd.merge(ff3, mom, on="Date")
ffData.rename(columns={ffData.columns[-1]: "Mom", "Mkt-RF": "Mkt_RF"}, inplace=True)
ffData.iloc[:, 1:] = ffData.iloc[:, 1:] / 100
ffData["Date"] = pd.to_datetime(ffData["Date"], format="%Y%m%d")

returns["Date"] = pd.to_datetime(returns["Date"], format="%m/%d/%Y")

stocks = ["AAPL", "MSFT", "BRK-B", "CSCO", "JNJ"]
to_reg = pd.merge(returns[["Date", "SPY"] + stocks], ffData, on="Date")

xnames = ['Mkt_RF', 'SMB', 'HML', 'Mom']

# OLS Regression for all Stocks
X = to_reg[xnames].values
Y = to_reg[stocks].values

Betas = []
for stock in stocks:
    lr = LinearRegression()
    lr.fit(X, to_reg[stock].values)
    Betas.append(lr.coef_)

Betas = np.array(Betas)

max_dt = to_reg["Date"].max()
min_dt = max_dt - pd.DateOffset(years=10)
to_mean = ffData[(ffData["Date"] >= min_dt) & (ffData["Date"] <= max_dt)]

# Historic daily factor returns
exp_Factor_Return = to_mean[xnames].mean().values

# Scale returns and covariance to geometric yearly numbers
stockMeans = np.log(1 + Betas @ exp_Factor_Return) * 255
covar = np.cov(np.log(1 + Y), rowvar=False) * 255

# Portfolio Volatility function
def pvol(w):
    return np.sqrt(w @ covar @ w)

# Component Standard Deviation function
def pCSD(w):
    p_vol = pvol(w)
    return w * (covar @ w) / p_vol

# Sum Square Error of cSD
def sseCSD(w):
    csd = pCSD(w)
    mCSD = np.sum(csd) / len(stocks)
    dCsd = csd - mCSD
    se = dCsd * dCsd
    return 1e5 * np.sum(se)

# Optimize to find Risk Parity weights
n = len(stocks)
initial_weights = np.ones(n) / n
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, None) for _ in range(n)]

result = minimize(sseCSD, initial_weights, constraints=constraints, bounds=bounds)
optimized_weights = result.x

RPWeights = pd.DataFrame({'Stock': stocks, 'Weight': optimized_weights, 'cEr': stockMeans * optimized_weights, 'CSD': pCSD(optimized_weights)})



In [93]:
import scipy.stats as stats

# Function for fitting t-distribution to the data
def fit_general_t(data):
    params = stats.t.fit(data)
    fitted_data = stats.t.rvs(params[0], params[1], params[2], size=len(data))
    return fitted_data, params

# Fit t-distributions to the returns
fitted_returns = []
params_list = []
for stock in stocks:
    fitted_data, params = fit_general_t(Y[:, stocks.index(stock)])
    fitted_returns.append(fitted_data)
    params_list.append(params)

U = np.column_stack(fitted_returns)

# Gaussian Copula simulation
nSim = 5000
corsp = stats.spearmanr(U).correlation
_simU = stats.norm.cdf(np.random.multivariate_normal(np.zeros(n), corsp, nSim))
simReturn = np.zeros_like(_simU)

for i in range(n):
    simReturn[:, i] = stats.t.ppf(_simU[:, i], *params_list[i])

# Internal ES function
def _ES(w):
    r = simReturn @ w
    return VaR.ES(r)

# Function for the component ES
def CES(w):
    n = len(w)
    ces = np.zeros(n)
    es = _ES(w)
    e = 1e-6
    for i in range(n):
        old = w[i]
        w[i] = w[i] + e
        ces[i] = old * (_ES(w) - es) / e
        w[i] = old
    return ces

# SSE of the Component ES
def SSE_CES(w):
    ces = CES(w)
    ces = ces - np.mean(ces)
    return 1e3 * (ces.T @ ces)

# Optimize to find RP based on Expected Shortfall
initial_weights = np.ones(n) / n
result_es = minimize(SSE_CES, initial_weights, constraints=constraints, bounds=bounds, method='SLSQP')
optimized_weights_es = result_es.x

ES_RPWeights = pd.DataFrame({'Stock': stocks, 'Weight': optimized_weights_es, 'cEr': stockMeans * optimized_weights_es, 'CES': CES(optimized_weights_es)})

print(ES_RPWeights)
print(RPWeights)

print("Degrees of freedom for all stocks:")
for i, stock in enumerate(stocks):
    print(stock, ":", params_list[i][0])



   Stock    Weight       cEr       CES
0   AAPL  0.147450  0.020898  0.001723
1   MSFT  0.140376  0.023888  0.001714
2  BRK-B  0.324363  0.035762  0.001759
3   CSCO  0.168639  0.023017  0.001727
4    JNJ  0.219172  0.015567  0.001714
   Stock    Weight       cEr       CSD
0   AAPL  0.154203  0.021855  0.022688
1   MSFT  0.142566  0.024261  0.022688
2  BRK-B  0.265160  0.029235  0.022688
3   CSCO  0.150786  0.020581  0.022688
4    JNJ  0.287285  0.020405  0.022688
Degrees of freedom for all stocks:
AAPL : 3209646.862325821
MSFT : 4.596292767770695
BRK-B : 21.748436506065993
CSCO : 4.5428908272455
JNJ : 9.428487611937966
